2024-10-16 2g gold
2024-10-22 200 USD; 20000 JPY

##### 環境設置

In [ ]:
import pandas as pd
import requests
from datetime import datetime
import os
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import json

# Excel 檔案名稱
FILENAME = "investment_tracking.xlsx"

# 初始化交易紀錄 DataFrame
if os.path.exists(FILENAME):
    transactions = pd.read_excel(FILENAME)
else:
    transactions = pd.DataFrame(columns=['日期', '投資類型', '投資項目', '數量_or_外幣金額', '買進匯率','價格(台幣)'])

In [ ]:
def get_gold_price():
    try:
        # Alpha Vantage 的 API URL
        # url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=XAUUSD&apikey=T4QJDQEMPG0ED9A3&datatype=json"
        url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=XAUUSD&apikey=IDC7T693IGDTA109&datatype=json"
        response = requests.get(url)
        data = response.json()
        
        # 將資料儲存成 JSON 檔案
        with open("gold_price_data.json", "w") as json_file:
            json.dump(data, json_file, indent=4)
        
        print("資料已成功儲存為 gold_price_data.json")
        return data
    except Exception as e:
        print("取得黃金價格時出現錯誤：", e)
        return None

# 呼叫函數並儲存資料
get_gold_price()

# 外匯匯率
def get_exchange_rate(base_currency):
    try:
        # url = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol={base_currency}&to_symbol=TWD&apikey=T4QJDQEMPG0ED9A3&outputsize=full"
        url = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol={base_currency}&to_symbol=TWD&apikey=IDC7T693IGDTA109&outputsize=full"
        response = requests.get(url)
        data = response.json()
        
        # 將資料儲存成 JSON 檔案
        with open(f"{base_currency}_exchange_rate.json", "w") as json_file:
            json.dump(data, json_file, indent=4)
        
        print(f"資料已成功儲存為 {base_currency}_exchange_rate.json")
        return data
    except Exception as e:
        print("取得黃金價格時出現錯誤：", e)
        return None

# 呼叫函數並儲存資料
currencies = ["USD", "EUR", "JPY", "AUD"]
for currency in currencies:
    get_exchange_rate(currency)

##### 黃金function

In [ ]:
# 取得黃金即時價格
def gold_price():
    try:
        with open("gold_price_data.json", "r") as json_file:
            data = json.load(json_file)

        # 獲取最近的日期
        latest_date = max(data["Time Series (Daily)"].keys())
        
        # 提取該日期的收盤價
        gold_price = float(data["Time Series (Daily)"][latest_date]["4. close"])
        
        print(f"黃金價格 (日期: {latest_date}): {gold_price} USD")
        return gold_price
    except Exception as e:
        print("取得黃金價格時出現錯誤：", e)
        return None

# 取得歷史黃金價格
def historical_gold_price(date):
    try:
        with open("gold_price_data.json", "r") as json_file:
            data = json.load(json_file)
      
        # 提取該日期的收盤價
        gold_price = float(data["Time Series (Daily)"][date]["4. close"])
        
        print(f"黃金價格 (日期: {date}): {gold_price} USD")
        return gold_price
    except Exception as e:
        print("取得黃金價格時出現錯誤：", e)
        return None


##### 外匯function

In [ ]:
# 取得當前匯率
def exchange_rate(base_currency):
    try:
        with open(f"{base_currency}_exchange_rate.json" "r") as json_file:
            data = json.load(json_file)

        if base_currency in data['rates']:
            return data['rates'][base_currency]
        else:
            print(f"無法取得 {base_currency} 的匯率。")
            return None
    except Exception as e:
        print("取得匯率時出現錯誤：", e)
        return None
    
# 取得歷史匯率
def historical_exchange_rate(base_currency,  date):
    try:
        with open(f"{base_currency}_exchange_rate.json", "r") as json_file:
            data = json.load(json_file)

        # 檢查是否成功獲取數據
        if 'Time Series FX (Daily)' in data:
            time_series = data['Time Series FX (Daily)']
            # 轉換日期格式
            
            if date in time_series:
                # 提取對應日期的匯率
                exchange_rate = float(time_series[date]['4. close'])
                return exchange_rate
            else:
                print(f"無法獲取 {base_currency} 和 TWD 的匯率，日期可能不存在。")
                return None
        else:
            print("無法獲取數據，請檢查匯率data或其他設置。")
            return None

    except Exception as e:
        print("取得匯率時出現錯誤：", e)
        return None



##### 主程式

In [ ]:
# 手動新增投資交易
def add_transaction():
    global transactions
    # 手動輸入資料
    年 = input("輸入交易年份: ")
    月 = input("輸入交易月份: ")
    日 = input("輸入交易日: ")
    投資類型 = input("輸入投資類型（Gold 或 Forex）: ").upper()
    投資項目 = input("輸入投資項目（如 Gold、USD、EUR 等）: ").upper()
    
    # 轉換日期格式
    日期 = datetime.strptime(f"{年}-{月}-{日}", '%Y-%m-%d')
    

    if 投資類型 == 'GOLD':
        # 輸入黃金數量
        while True:
            try:
                數量_or_外幣金額 = float(input("輸入黃金數量 (克數): "))

                # 取得歷史黃金價格，且由於原本重量以盎司計算將其轉換為克
                gold_price_usd = historical_gold_price(日期)/31.1035
                if gold_price_usd is not None:
                    # 獲取歷史匯率
                    exchange_rate = historical_exchange_rate('USD', 日期)
                    if exchange_rate is not None:
                        # 將黃金價格轉換為台幣
                        黃金價格 = gold_price_usd * exchange_rate
                        價格 = 黃金價格 * 數量_or_外幣金額
                        break
                print("無法獲取黃金價格，請檢查日期或匯率data。")
                return
            except ValueError:
                print("請輸入一個有效的數字。")
    
    elif 投資類型 == 'FOREX':
        # 輸入外幣數量
        while True:
            try:
                數量_or_外幣金額 = float(input("輸入買入外幣金額: "))  # 外幣數量由用戶輸入
                # 使用指定日期的匯率
                匯率 = historical_exchange_rate(投資項目, 日期)
                if 匯率 is None:
                    print("無法獲取匯率，請檢查日期或匯率data。")
                    return
                
                價格 = 匯率 * 數量_or_外幣金額  # 使用匯率計算價格（台幣）
                break
            except ValueError:
                print("請輸入一個有效的數字。")
    
    else:
        print("無效的投資類型！")
        return

    # 建立新交易 DataFrame
    transaction = pd.DataFrame({
        '日期': [日期],
        '投資類型': [投資類型],
        '投資項目': [投資項目],
        '數量_or_外幣金額': [數量_or_外幣金額],
        '買進匯率': [匯率],
        '買進價格(台幣)': [價格]
    })
    
    transactions = pd.concat([transactions, transaction], ignore_index=True)

# 儲存至 Excel 並追加資料
def save_to_excel():
    if os.path.exists(FILENAME):
        # 如果 Excel 檔案已存在，追加新資料
        with pd.ExcelWriter(FILENAME, mode='a', if_sheet_exists='overlay', engine='openpyxl') as writer:
            transactions.to_excel(writer, index=False, sheet_name='Sheet1')
    else:
        # 如果 Excel 檔案不存在，創建新檔案
        transactions.to_excel(FILENAME, index=False)

# 手動新增交易
add_transaction()

# 儲存更新後的投資紀錄
save_to_excel()

print("投資紀錄已儲存至 Excel 檔案。")


##### 測試

In [ ]:
# test
# get_gold_price()
# get_historical_gold_price('2024-10-16')
# get_exchange_rate('USD')
# get_historical_exchange_rate('USD', '2024-10-16')

# test = get_historical_exchange_rate('USD',  '2024/10/16')
test = get_historical_gold_price('2024/10/16') # * get_historical_exchange_rate('USD', '2024/10/16') /31.1035

print(test)